In [ ]:
# from torchsummary import summary
# from torchviz import make_dot
import albumentations as A
import mlflow
import torch.nn as nn
import torch.optim as optim
import tqdm
from albumentations.pytorch import ToTensorV2
from torch.optim.lr_scheduler import PolynomialLR
import cv2
from DiceBCELoss import DiceBCELoss
from model import *
from utils import *

In [ ]:
EXPERIMENT_NAME = "CS679_Project"

In [ ]:
# experiment_id = mlflow.create_experiment(f"{EXPERIMENT_NAME}")
experiment = mlflow.get_experiment_by_name(f"{EXPERIMENT_NAME}")
mlflow.set_experiment(f"{EXPERIMENT_NAME}")

In [ ]:
lr = 1e-4
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 4
epochs = 100
pin_memory = True

# Data

Path: \
stenosis_train = ARCADE('dataset/stenosis/train')\
syntax__test = ARCADE('dataset/syntax_/test')\
syntax__val = ARCADE('dataset/stenosis/val')\
syntax_train = ARCADE('dataset/syntax/train')\
syntax_test = ARCADE('dataset/syntax/test')\
syntax_val = ARCADE('dataset/syntax/val')

In [ ]:
sample = ARCADE('dataset/stenosis/train')[10]
image = sample[0].reshape((512, 512))
mask = sample[1].reshape((512, 512))

plot_image_with_mask(image, mask)

In [ ]:
image.shape

In [ ]:
mask.shape

In [ ]:
# load the dataset and apply transformations, for validation we don't need to apply any transformations

train_transform = A.Compose([

    A.VerticalFlip(p=0.5),

    A.HorizontalFlip(p=0.5),

    A.ShiftScaleRotate(

        shift_limit=0.3,

        scale_limit=0.5,

        rotate_limit=30,

        border_mode=cv2.BORDER_REFLECT,

        p=0.5

    ),

    A.Affine(shear=5, p=0.5),

    A.Perspective(scale=(0.001, 0.001), p=0.5),

    ToTensorV2()

])


val_transform = A.Compose([

    ToTensorV2()  # This ensures the output is in the format PyTorch expects

])


stenosis_train, stenosis_test, stenosis_val = get_loaders('dataset/stenosis/train', 'dataset/stenosis/test',

                                                          'dataset/stenosis/val', batch_size, train_transform,

                                                          val_transform, val_transform)


# syntax_train, syntax_test, syntax_val = get_loaders('dataset/syntax/train', 'dataset/syntax/test', 'dataset/syntax/val', batch_size, train_transform,

#                                                           val_transform, val_transform)

# Model

In [ ]:
# model = UNetPlusPlus()
# summary(model, (1, 512, 512))

In [ ]:
# os.environ["PATH"] += os.pathsep + 'F:/Program Files/Graphviz/bin/'
# x = torch.randn(1, 1, 512, 512)
# y = model(x)
# make_dot(y.mean(), params=dict(model.named_parameters()), show_attrs=True, show_saved=True).render("rnn_torchviz", format="png")

# Training

In [ ]:
model = UNetPlusPlus().to(device)

# define the optimizer and loss function
criterion = DiceBCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scaler = torch.amp.GradScaler()

best_f1, best_iou, best_dsc = 0.0, 0.0, 0.0

# Using mlflow to log the model and parameters
mlflow.start_run()
mlflow.log_param("learning_rate", lr)
mlflow.log_param("epochs", epochs)
mlflow.log_param("batch_size", batch_size)
mlflow.log_param("optimizer", "Adam")
mlflow.log_param("Scaler", "GradScaler")
mlflow.log_param("loss_function", "DiceBCELoss")

In [ ]:
for epoch in tqdm.trange(epochs):
    # loop = tqdm.tqdm(stenosis_train, leave=True, position=0)
    # for batch_idx, (data, targets) in enumerate(loop):
    for batch_idx, (data, targets) in enumerate(stenosis_train):
        data = data.to(device=device)
        targets = targets.float().to(device=device)

        # forward
        predictions = model(data)

        # if deep supervision is used, the predictions will be a list of tensors
        if model.deep_supervision:
            loss = 0
            for i in range(len(predictions)):
                loss += criterion(predictions[i], targets)
        else:
            loss = criterion(predictions, targets)

        # log metrics to mlflow
        mlflow.log_metric("Loss", loss.item(), step=epoch *
                          batch_size + batch_idx + 1)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    # update metrics
    avg_f1, avg_iou, avg_dsc = track_metric(stenosis_val, model, device=device)

    mlflow.log_metric("Avg F1", avg_f1, step=epoch)
    mlflow.log_metric("Avg IoU", avg_iou, step=epoch)
    mlflow.log_metric("Avg DSC", avg_dsc, step=epoch)

    # if we get a better f1 score, save the model
    if avg_f1 > best_f1:
        best_f1 = avg_f1
        best_iou = avg_iou
        best_dsc = avg_dsc
        print(f"Best F1: {best_f1}, IoU: {best_iou}, DSC: {best_dsc}")
        torch.save(model, "./checkpoint/best.pth".format(epoch))
        # mlflow.log_artifact("./checkpoint/best.pth".format(epoch))
    else:
        print(f"F1: {avg_f1}, IoU: {avg_iou}, DSC: {avg_dsc}")
        torch.save(model, "./checkpoint/last.pth".format(epoch))
        # mlflow.log_artifact("./checkpoint/last.pth".format(epoch))
mlflow.end_run()

In [ ]:
mlflow.end_run()

In [ ]:
model = torch.load("./checkpoint/best.pth")

In [ ]:
avg_f1, avg_iou, avg_dsc = track_metric(
    stenosis_test, model, device=device, post_processing=False)

print(f"Without post-processing: {avg_f1}, {avg_iou}, {avg_dsc}")

In [ ]:
avg_f1, avg_iou, avg_dsc = track_metric(
    stenosis_test, model, device=device, post_processing=600)

print(f"With post-processing: {avg_f1}, {avg_iou}, {avg_dsc}")